# Shortest path

In [28]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import json


## Récuperer toutes les gares de France

In [29]:
dfStop = pd.read_csv('../data_sncf/stops.txt', sep=",")
dfStop.shape

(9176, 9)

# New way of calc

## Import file

In [30]:
dfTimeTables = pd.read_csv('../data_sncf/timetables.csv', sep="\t")
dfTimeTables.dropna().head()


,trip_id,trajet,duree
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149


## Split str to get departure and arrival

## Vérification de l'intégrité des données

In [31]:
# Vérifier si la colonne 'trajet' contient des valeurs nulles
if dfTimeTables['trajet'].isnull().any():
    print("La colonne 'trajet' contient des valeurs nulles.")
else:
    print("La colonne 'trajet' ne contient pas de valeurs nulles.")

# Vérifier si tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '
if (dfTimeTables['trajet'].str.count(' - ') == 1).all():
    print("Tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '.")
else:
    print("Certains éléments de la colonne 'trajet' n'ont pas exactement une occurrence de ' - '.")

dfTimeTables[dfTimeTables['trajet'].str.count(' - ') != 1]

La colonne 'trajet' ne contient pas de valeurs nulles.
Tous les éléments de la colonne 'trajet' ont exactement une occurrence de ' - '.


,trip_id,trajet,duree


## Split de tous les trajets pour avoir le départ et l'arrivée

In [32]:
dfTimeTables[['depart', 'arrivee']] = dfTimeTables['trajet'].str.split(' - ', expand=True)

dfTimeTables.head()

,trip_id,trajet,duree,depart,arrivee
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138,Gare de Le Havre,Gare de Paris-St-Lazare
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145,Gare de Dieppe,Gare de Paris-St-Lazare
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97,Gare de Paris-St-Lazare,Gare de Rouen-Rive-Droite
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194,Gare de Cherbourg,Gare de Paris-St-Lazare
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149,Gare de Caen,Gare de Paris-St-Lazare


## Création du graph avec tous les voisins

In [33]:
def get_neighborv1():
    graph = {}
    # graph[dfTimeTables["depart"].values[i]] = {}

    for i in range(len(dfTimeTables["depart"])):
        try:
            depart = dfTimeTables["depart"].values[i]
            arrivee = dfTimeTables["arrivee"].values[i]
            temps = dfTimeTables["duree"].values[i]
            if depart not in graph:
                graph[depart] = {}
            graph[depart] = {**graph[depart], arrivee: int(temps*60)}

        except Exception as e:
            print("Une erreur s'est produite: {}".format(e))

        # Récupération de tous les noms de gares
        # On part du principe qu'un aller à forcément un retour, on peut donc ne
        # rechercher que dans les départs

    # # create json object from dictionary
    json_values = json.dumps(graph)
    print(json_values)
    #


    # Create a file name with the current date and time
    file_name = "data_csv_.json"

    # # open file for writing, "w"
    f = open(file_name, "w")
    #
    # # write json object to file
    f.write(json_values)
    #
    # # close file
    f.close()
    print(graph)


get_neighborv1()

{"Gare de Le Havre": {"Gare de Paris-St-Lazare": 8280, "Gare de Montivilliers": 900, "Gare de Rolleville": 1560, "Gare de Rouen-Rive-Droite": 3660}, "Gare de Dieppe": {"Gare de Paris-St-Lazare": 8700, "Gare de Rouen-Rive-Droite": 3780}, "Gare de Paris-St-Lazare": {"Gare de Rouen-Rive-Droite": 5820, "Gare de Trouville-Deauville": 8820, "Gare de Vernon-Giverny": 3600, "Gare de Serquigny": 6300}, "Gare de Cherbourg": {"Gare de Paris-St-Lazare": 11640}, "Gare de Caen": {"Gare de Paris-St-Lazare": 8940, "Gare de Le Mans": 6660, "Gare de Tours": 10920, "Gare de Rouen-Rive-Droite": 6000, "Gare de Rennes": 10200, "Gare de Cherbourg": 10440, "Gare de Lisieux": 1980, "Gare de Coutances": 4500, "Gare de St-L\u00f4": 3060, "Gare de Dol": 10020, "Gare de Granville": 6180}, "Gare de Granville": {"Gare de Paris-Montp.3-Vaug.": 12060, "Gare de Paris-Montparnasse 1-2": 11700, "Gare de Lison": 5040, "Vire-Centre": 4500, "Gare de Rennes": 6600}, "Gare de Dreux": {"Gare de Granville": 8520, "Gare de L'Aig

In [34]:
dfStop = dfStop[dfStop['stop_id'].str.contains('StopPoint:OCETrain')]
dfStop.shape

(2447, 9)

In [35]:
allGare = dfStop['stop_name'].unique()
print(allGare)


['Gare de Paris-St-Lazare' 'Gare de Rouen-Rive-Droite' 'Gare de Montville'
 ... 'Gare de Novillars' 'Gare de Deluz' 'Gare de Laissey']


## Extractons nos voisins

In [36]:
dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")

/var/folders/0l/4rhc85qd4kl41w3sd26dml3h0000gn/T/ipykernel_44772/3309677073.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")


In [37]:
def str_to_time(time: str):
    time_split = time.split(':')
    if time_split[0] > '23':
        time_split[0] = str(int(time_split[0]) - 24)
    time = f'{time_split[0]}:{time_split[1]}:{time_split[2]}'
    time_object = datetime.strptime(time, '%H:%M:%S')
    return time_object

In [38]:
def getVoisins(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for i in range(len(trajets)):
        # on recupere la position de la gare dans le trajet
        stopSequence = trajets['stop_sequence'].values[i]
        # on recupere le tripID du trajet
        tripId = trajets['trip_id'].values[i]
        # on recupere les times de notre gare
        departureTimeGare = str_to_time(trajets['departure_time'].values[i])
        arivalTimeGare = str_to_time(trajets['arrival_time'].values[i])

        try:
            # on recupere la gare précédente
            voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence - 1)]
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = str_to_time(voisin1.departure_time.values[0])
            # on recupere le nom de la gare
            voisin1 = dfStop[dfStop['stop_id'] ==
                             voisin1.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue

        try:
            # on recupere la gare suivante
            voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence + 1)]
            # on recupere le departure time de la gare suivante
            arrivalTimeVoisin2 = str_to_time(voisin2.departure_time.values[0])
            # on recupere le nom de la gare
            voisin2 = dfStop[dfStop['stop_id'] ==
                             voisin2.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue

    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare


In [39]:
def get_neighborv2():
    graph = {}

    # for i in tqdm(range(len(allGare))):
    for i in tqdm(range(10)):
        depart = dfStop["stop_name"].values[i]
        # On récupère l'id/stop id de notre gare
        stopID = dfStop[(dfStop.stop_name == allGare[i])].stop_id.values[0]

        # On récupère les trajets qui passent par cette gare
        trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]

        # On parcours ces trajets
        for j in range(len(trajets)):
            # on recupere la position de la gare dans le trajet
            stopSequence = trajets['stop_sequence'].values[j]
            print(stopSequence)
            # on recupere le tripID du trajet
            tripId = trajets['trip_id'].values[j]
            # graph[depart] = {**graph[depart], voisin: 1}

            # on recupere les times de notre gare
            # departureTimeGare = str_to_time(trajets['departure_time'].values[j])
            # arivalTimeGare = str_to_time(trajets['arrival_time'].values[j])

        # on recupere les trajets qui passent par cette gare
        trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
        try:
            print("le")
            # depart = dfStop['stop_name'].values[i]
            # if depart not in graph:
            #     graph[depart] = {}
            # print(trajets)
            # for j in range(len(trajets)):
            #
            #     graph[depart] = {**graph[depart], trajets[j]: 1}
        except Exception as e:
            print("Une erreur s'est produite: {}".format(e))


#     for i in range(len(dfTimeTables["depart"])):
#         try:
#             depart = dfTimeTables["depart"].values[i]
#             arrivee = dfTimeTables["arrivee"].values[i]
#             temps = dfTimeTables["duree"].values[i]
#             if depart not in graph:
#                 graph[depart] = {}
#             graph[depart] = {**graph[depart], arrivee: int(temps)}
#
#         except Exception as e:
#             print("Une erreur s'est produite: {}".format(e))
#
#         # Récupération de tous les noms de gares
#         # On part du principe qu'un aller à forcément un retour, on peut donc ne
#         # rechercher que dans les départs
#
#     # # create json object from dictionary
#     json_values = json.dumps(graph)
#     print(json_values)
#     #
#     now = datetime.datetime.now()
#
#     # Create a file name with the current date and time
#     date_string = now.strftime("%Y-%m-%d_%H-%M-%S")
#     file_name = "data_" + date_string + ".json"
#
#     # # open file for writing, "w"
#     f = open(file_name, "w")
#     #
#     # # write json object to file
#     f.write(json_values)
#     #
#     # # close file
#     f.close()
    print(graph)

#
# get_neighborv2()

In [40]:
graph = {}
# for i in tqdm(range(len(allGare))):
#     graph[allGare[i]] = get_neighborv1(allGare[i])
get_neighborv1()
print(graph)



{"Gare de Le Havre": {"Gare de Paris-St-Lazare": 8280, "Gare de Montivilliers": 900, "Gare de Rolleville": 1560, "Gare de Rouen-Rive-Droite": 3660}, "Gare de Dieppe": {"Gare de Paris-St-Lazare": 8700, "Gare de Rouen-Rive-Droite": 3780}, "Gare de Paris-St-Lazare": {"Gare de Rouen-Rive-Droite": 5820, "Gare de Trouville-Deauville": 8820, "Gare de Vernon-Giverny": 3600, "Gare de Serquigny": 6300}, "Gare de Cherbourg": {"Gare de Paris-St-Lazare": 11640}, "Gare de Caen": {"Gare de Paris-St-Lazare": 8940, "Gare de Le Mans": 6660, "Gare de Tours": 10920, "Gare de Rouen-Rive-Droite": 6000, "Gare de Rennes": 10200, "Gare de Cherbourg": 10440, "Gare de Lisieux": 1980, "Gare de Coutances": 4500, "Gare de St-L\u00f4": 3060, "Gare de Dol": 10020, "Gare de Granville": 6180}, "Gare de Granville": {"Gare de Paris-Montp.3-Vaug.": 12060, "Gare de Paris-Montparnasse 1-2": 11700, "Gare de Lison": 5040, "Vire-Centre": 4500, "Gare de Rennes": 6600}, "Gare de Dreux": {"Gare de Granville": 8520, "Gare de L'Aig

In [41]:
# load json module
# import json
#
# # create json object from dictionary
print(graph)
json_data = json.dumps(graph)
#
# # open file for writing, "w"
f = open("le.json", "w")
#
# # write json object to file
f.write(json)
#
# # close file
f.close()

{}


TypeError: write() argument must be str, not module